In [ ]:
import tensorflow
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
import numpy as np

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)

x_train = tensorflow.image.resize(x_train, (48, 48))
x_test = tensorflow.image.resize(x_test, (48, 48))
print(x_train.shape)

x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = np.stack((x_train,) * 3, axis=-1)
x_test = np.stack((x_test,) * 3, axis=-1)
print(x_train.shape)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    shear_range=0.1,
    height_shift_range=0.1,
    width_shift_range=0.1
)

datagen.fit(x_train)
augmented_train = datagen.flow(x_train, y_train, batch_size=32)

datagen.fit(x_test)
augmented_test = datagen.flow(x_test, y_test, batch_size=32)

In [ ]:
model1 = Sequential()
model1.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
model1.add(Dense(512, activation='relu'))
model1.add(Dense(10, activation='softmax'))

model1.layers[0].trainable=False
model1.summary()

In [ ]:
model2 = Sequential()
model2.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
model2.add(Dense(512, activation='relu'))
model2.add(Dense(10, activation='softmax'))

model2.layers[0].trainable=False
model2.summary()

In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model1.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(augmented_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

In [ ]:
out11 = model1.evaluate(x_test, y_test)

print("Model Accuracy: %.4f" % out11[1])
print("Model loss: %.4f" % out11[0])

out21 = model2.evaluate(x_test, y_test)

print("Model Accuracy: %.4f" % out21[1])
print("Model loss: %.4f" % out21[0])

In [ ]:
out12 = model1.evaluate(augmented_test)
# print(out12)

print("Model Accuracy: %.4f" % out12[1])
print("Model Loss: %.4f" % out12[0])

out21 = model2.evaluate(augmented_test)

print("Model Accuracy: %.4f" % out21[1])
print("Model Loss: %.4f" % out21[0])